In [11]:
import MovementSimulator_0_1 as ms
import cv2
import path_tracking_ini_functions as ptif
import time
import math
import numpy as np

In [17]:
from random import gauss
def reaction_f(action):
    x_a,y_a,f_a = action
    
    rx = gauss(0,1)/1.3
    ry = gauss(0,1)/1.3
    
    x_re = (x_a + ry*y_a) * f_a
    y_re = (y_a - rx*x_a) * f_a
    
    return x_re, y_re

def reaction_0(action):
    
    return 0,0

In [5]:
def generate_field_example():
    particle1 = ms.SimulatedParticleClass((30,30),reaction_f,25)

    
    target1 = ms.SimulatedTargetsClass((125,75), reaction_0,20)
    target2 = ms.SimulatedTargetsClass((180,250),reaction_0, 30)
    target3 = ms.SimulatedTargetsClass((330,310), reaction_0,25)
    target4 = ms.SimulatedTargetsClass((200,200), reaction_0,20)
    target5 = ms.SimulatedTargetsClass((300,100),reaction_0, 30)
    target6 = ms.SimulatedTargetsClass((300,40), reaction_0,25)
    
    particles_dict = {'p1':particle1}
    targets_dist = {'t1':target1, 't2':target2, 't3':target3, 't4':target4, 't5':target5, 't6':target6}
    borders = (0,0,480,640)
    
    RealField = ms.SimulatedFieldClass(particles_dict,targets_dist,borders)
    return RealField

In [6]:
def vector_len(x,y):
    length = math.sqrt(x**2 + y**2)
    return length

In [7]:
def show_path(image, path):
    
    for i in range(len(path)-1):
        image = cv2.line(image,path[i],path[i+1],(150,150,150))
    return image

In [18]:
# main tracker-controller cell


RF = generate_field_example()
image = RF.generate_image(width = 640, height = 480)
kernel = np.ones((5,5),np.float32)/25
image = cv2.filter2D(image,-1,kernel)

# cv2.imshow("Tracking", image)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

tracking = ptif.PathFinderAutoClass(image)
path, bound = tracking.initiate_path_tracking(compress = False)

tracker = cv2.legacy.TrackerCSRT_create()
ok = tracker.init(image, bound)

# if the path was found
if len(path)>0:
    # current particle coordinates
    x1,y1 = (bound[0]+bound[2]//2, bound[1]+bound[3]//2)
    path_point = 0
    
    time_step = 4
    force = 500 
    
    stop = False
        
    while stop != True:
        
        x_p,y_p = path[path_point]
        
        action_x, action_y = (x_p - x1,y_p - y1)
        action_x_n, action_y_n = (action_x/vector_len(action_x, action_y),action_y/vector_len(action_x, action_y))
        action = (action_x_n,action_y_n,force)
        
        
        if vector_len(action_x, action_y) < 7:
            path_point = path_point + 1
        if (path_point > (len(path)-1)):
            stop = True
 
        for i in range(5):
            ok = RF.next_time_step(action, time_step)
        
        if ok > 0:
            stop = True
                       
        image = RF.generate_image(width = 640, height = 480)
        image = cv2.filter2D(image,-1,kernel)
        ok, bound = tracker.update(image)
        
        if ok:
            p1 = (int(bound[0]), int(bound[1]))
            p2 = (int(bound[0] + bound[2]), int(bound[1] + bound[3]))
            image = cv2.rectangle(image, p1, p2, (150,150,150), 2, 1)
                   
        x1,y1 = (bound[0]+bound[2]//2, bound[1]+bound[3]//2)
        
    
        
        image = show_path(image, path)
        cv2.imshow("Tracking", image)
        cv2.waitKey(200)
else:
    print('No path was found')
        
cv2.waitKey(0)
cv2.destroyAllWindows()    
    
    
    
    

(0, 0, 640, 480)
Left click on the ActiveParticle. When done, please press any key
(26, 32)
Left click on the Target. When done, please press any key
(579, 212)
6
-1


In [10]:
show_path(image,path)

28 28
28 288
104 288
104 296
108 296
108 308
112 308
112 312
116 312
116 316
120 316
120 320
136 320
136 324
260 324
260 360
264 360
264 368
268 368
268 372
276 372
276 376
504 376
504 388
